In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
bpc=pd.read_excel("BPC_Ingredientes.xlsx")
an3=pd.read_excel('annex_iii_inventory_en_PARA CARGAR en 03.xls')

In [3]:
an3

,Name,EC / List no.,CAS no.,Criteria(Indication)
0,Dexamethasone,200-003-9,50-02-2,Suspected mutagen#Suspected toxic for reproduc...
1,Cortisone 21-acetate,200-006-5,50-04-4,Suspected carcinogen#Suspected hazardous to th...
2,Phenobarbital,200-007-0,50-06-6,Suspected carcinogen#Suspected hazardous to th...
3,Mitomycin,200-008-6,50-07-7,Suspected carcinogen#Suspected hazardous to th...
4,Hexobarbital sodium,200-009-1,50-09-9,Suspected hazardous to the aquatic environment...
...,...,...,...,...
64894,4-Fluorophthalic Anhydride,-,319-03-9,Suspected acutely toxic via the oral route#Sus...
64895,Ethyl 9-Bromononanoate,-,28598-81-4,Suspected acutely toxic via the oral route#Sus...
64896,2-dodecoxyethyl hydrogen sulfate,-,9004-82-4,Suspected acutely toxic via the oral route#Sus...
64897,methyl benzenesulfonate,-,80-18-2,Suspected acutely toxic via the oral route#Sus...


In [4]:
names=an3['Name'].apply(lambda x: x.lower()).to_list()
ec=an3['EC / List no.'].to_list()
cas=an3['CAS no.'].to_list()



In [5]:
blurb='European Commission Legislation (Regulation (EC) N° 1223/2009 y 2019/831, Annexes I to VI and all the amendments to the cosmetics regulation).'

In [6]:
bpc_mapped=[]
for n,node in bpc.iterrows():
    for sinonimo in node['Synonyms.formatx'].split(';'):
        buffer=[]
        query=an3[
            (an3['Name'] == sinonimo) |\
            (an3['EC / List no.'] == node['EC No']) |\
            (an3['CAS no.'] == node['CAS No'])
        ]
        if not query.empty:
            node['modificado']=True
            ttm=query['Criteria(Indication)'].values[0]
            if node['Anexo.iii.Criteria'] == '-':
                node['Anexo.iii.Criteria']=ttm
                node['Info para Reporte']=node['Info para Reporte']
                node['Citas']= node['Citas'] + '{};'.format(blurb)
            if node['CAS No']=='-' and query['CAS no.'].values[0]!='-':
                node['CAS No'] = query['CAS no.'].values[0]
                node['Synonyms.formatx']=node['Synonyms.formatx'] + '{};'.format(query['CAS no.'].values[0])
            if node['EC No']=='-' and query['EC / List no.'].values[0] != '-':
                node['EC No'] = query['EC / List no.'].values[0]
                node['Synonyms.formatx']=node['Synonyms.formatx'] + '{};'.format(query['EC / List no.'].values[0])
            buffer=node    
           
            break
            
        else:
            buffer=node
        
    bpc_mapped.append(buffer)


            
            

In [7]:
df_an3_no_dd=pd.concat(bpc_mapped, axis=1).T

In [14]:
df_an3_no_dd.to_excel("prueba_es_este.xlsx")

In [9]:
df_an3_2=pd.concat(bpc_mapped, axis=1).T.drop_duplicates(subset=['Synonyms.formatx'])


In [17]:
risky=df_an3_no_dd[df_an3_no_dd['Total.Risk']>=2]
risky_itr=risky[risky['Info para Reporte']!='-']
risky_final=risky_itr[risky_itr.Citas!='-']
risky_final.to_excel("risky_prueba.xlsx")

In [19]:
risky.to_excel("rararara.xlsx")

In [11]:
risky_final.to_excel('bpc_con_an3_itr_citas.xlsx')

In [12]:
extractos=pd.read_excel('extractos.xlsx')
extractos.columns=['name', 'Info para Reporte', 'Citas']

In [13]:
rows_con_extractos=[]
for n,node in bpc.iterrows():
    for sinonimo in node['Synonyms.formatx'].split(';'):
        query=extractos[
            (extractos['name'] == sinonimo)
        ]
        if query.empty:
            rows_con_extractos.append(node)
        else:
            print('salte aca')
            node['modificado']=True
            ipr=query['Info para Reporte'].values[0]
            citas=query['Citas'].values[0]
            if node['Info para Reporte'] == '-':
                # node['Info para Reporte']=ipr
                node['Info para Reporte']=node['Info para Reporte']
                node['Citas']=node['Citas']+ ' {}'.format(citas)
            rows_con_extractos.append(node)

            break
            